In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import os 

import tensorflow as tf 
from tensorflow import keras
from keras.models import Sequential, Model
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras import regularizers, optimizers
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler

C:\Users\Erick Rocha\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Erick Rocha\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Erick Rocha\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Erick Rocha\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:5

In [2]:
DATASET_PATH = "C:/Users/Erick Rocha/Desktop/Analise Economica/Trabalho_Final/"
print(os.listdir(DATASET_PATH))

['.ipynb_checkpoints', 'images', 'styles.csv', 'Trabalho_XP.ipynb', 'Trab_Analise_pred_Avancada_final.ipynb']


In [3]:
df = pd.read_csv(DATASET_PATH + "styles.csv", error_bad_lines=False)
df['image'] = df.apply(lambda row: str(row['id']) + ".jpg", axis=1)
df = df.sample(frac=1).reset_index(drop=True)
df.head(10)

b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11\nSkipping line 33020: expected 10 fields, saw 11\nSkipping line 35748: expected 10 fields, saw 11\nSkipping line 35962: expected 10 fields, saw 11\nSkipping line 37770: expected 10 fields, saw 11\nSkipping line 38105: expected 10 fields, saw 11\nSkipping line 38275: ex

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,12162,Women,Apparel,Bottomwear,Leggings,Green,Fall,2011.0,Ethnic,Aurelia Women Solid Blue Leggings,12162.jpg
1,57661,Women,Personal Care,Nails,Nail Polish,Gold,Spring,2017.0,Casual,Lotus Herbals Gold Mist Nail Polish 972,57661.jpg
2,21465,Women,Apparel,Topwear,Sweaters,Red,Winter,2012.0,Casual,Roxy Women Red Sweater,21465.jpg
3,3713,Men,Apparel,Topwear,Tshirts,Green,Summer,2011.0,Casual,Lee Men Green Polo T-shirt,3713.jpg
4,10105,Women,Accessories,Watches,Watches,Steel,Winter,2016.0,Casual,Fastrack Women Analogue Leather Steel Watches,10105.jpg
5,47121,Boys,Footwear,Shoes,Casual Shoes,Navy Blue,Winter,2012.0,Casual,Marvel Navy Blue Boys Casual Shoes,47121.jpg
6,34886,Women,Accessories,Bags,Handbags,Multi,Summer,2012.0,Casual,Envirosax Women Multi Coloured Bag,34886.jpg
7,37511,Boys,Apparel,Socks,Booties,Yellow,Summer,2012.0,Casual,Madagascar 3 Infant Boys Yellow Booties,37511.jpg
8,3523,Men,Apparel,Topwear,Tshirts,Navy Blue,Summer,2011.0,Casual,Myntra Men's I Don't Suffer From Navy Blue T-s...,3523.jpg
9,23617,Women,Personal Care,Fragrance,Fragrance Gift Set,Orange,Spring,2017.0,Casual,Nike Fragrances Women Orange Blue Fantasy Pack...,23617.jpg


In [4]:
print('Treino: ', df.shape)

Treino:  (44424, 11)


In [5]:
df.masterCategory.unique()

array(['Apparel', 'Personal Care', 'Accessories', 'Footwear',
       'Free Items', 'Sporting Goods', 'Home'], dtype=object)

In [6]:
from keras_preprocessing.image import ImageDataGenerator

image_generator = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        fill_mode='nearest'
)

training_generator = image_generator.flow_from_dataframe(
    dataframe=df,
    directory=DATASET_PATH + "images",
    x_col="image",
    y_col="masterCategory",
    target_size=(96,96),
    batch_size=32,
    subset="training"
)

test_generator = image_generator.flow_from_dataframe(
    dataframe=df,
    directory=DATASET_PATH + "images",
    x_col="image",
    y_col="masterCategory",
    target_size=(96,96),
    batch_size=32,
    subset="validation"
)

classes = len(training_generator.class_indices)

C:\Users\Erick Rocha\anaconda3\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:273: UserWarning: Found 5 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 35536 validated image filenames belonging to 7 classes.
Found 8883 validated image filenames belonging to 7 classes.


In [7]:
from keras import models
from keras import layers

network = models.Sequential()

network.add(Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=(96, 96, 3)))
network.add(BatchNormalization())

network.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
network.add(BatchNormalization())
network.add(Dropout(0.25))

network.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
network.add(MaxPooling2D(pool_size=(2, 2)))
network.add(Dropout(0.25))

network.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
network.add(MaxPooling2D(pool_size=(2, 2)))
network.add(BatchNormalization())
network.add(Dropout(0.25))

network.add(Flatten())
network.add(Dense(512, activation='relu'))
network.add(BatchNormalization())
network.add(Dropout(0.5))
network.add(Dense(128, activation='relu'))
network.add(BatchNormalization())
network.add(Dropout(0.5))
network.add(Dense(7, activation='softmax'))

In [8]:
network.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [9]:
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 96, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 96, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 96, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 96, 96, 32)        128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 96, 96, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 64)       

In [ ]:
history = network.fit_generator(
    generator=training_generator,
    steps_per_epoch=training_generator.n/training_generator.batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.n/test_generator.batch_size,
    epochs=20 
)


Epoch 1/20
 240/1110 [=====>........................] - ETA: 1:24:20 - loss: 1.4031 - accuracy: 0.5796